In [1]:
import sys
sys.path.append("../../../")

In [2]:
from onsager import crystal, supercell, cluster
import numpy as np
import scipy as sp
import itertools
import Transitions
import Cluster_Expansion
import MC_JIT
import time
from tqdm import tqdm
from numba import jit, int64, float64

In [3]:
# Function to get symmtery info by full cluster translation
def ClusterSymmInfo(state, VclusExp, Nsym):
    
    allSpCl = [spcl for clList in VclusExp.SpecClusters for spcl in clList]
    symCountsTotal = np.zeros(Nsym, dtype=int)
    symCountSites = np.zeros((VclusExp.Nsites, Nsym), dtype=int)

    for siteInd in range(VclusExp.Nsites):
        ci, R = VclusExp.sup.ciR(siteInd)

        # Now translate the clusters about with this translation
        for spcl in allSpCl:
            offcount = 0 # count if the cluster is on for this translation
            for (site, spec) in spcl.SiteSpecs:
                siteNew = VclusExp.sup.index(site.R + R, site.ci)
                if state[siteNew] != spec:
                    offcount += 1

            if offcount == 0:
                # get the symmetry class
                symclass = VclusExp.clust2SpecClus[spcl][0] 
                # 0th element - symmetry class
                # 1st element - index into the symmetry class's list

                # Increment On count for symmetry class
                symCountsTotal[symclass] += 1

                # Increment symmetry count for each site
                for (site, spec) in spcl.SiteSpecs:
                    SupSite = VclusExp.sup.index(site.R + R, site.ci)[0]
                    symCountSites[SupSite, symclass] +=1
    
    return symCountsTotal, symCountSites

## Set up the vector cluster expander

In [4]:
import pickle

# See if a cluster expander already exists
try:
    with open("ClusterExpander.pkl", "rb") as fl:
        print("Pre-created Cluster expander found")
        VclusExp = pickle.load(fl)

# Otherwise make one
except:
    N_units = 8 # No. of unit cells along each axis in the supercell
    NSpec = 3
    MaxOrder = 3
    MaxOrderTrans = 3

    a0 = 1.0
    cut = 1.01*a0*np.sqrt(3)/2  # Nearest neighbor cutoff

    crys = crystal.Crystal.BCC(a0, chemistry="A")
    jnetBCC = crys.jumpnetwork(0, cut)

    superlatt = N_units * np.eye(3, dtype=int)
    superBCC = supercell.ClusterSupercell(crys, superlatt)
    # Since the "spectator" tuple in the argument is left empty, all
    # sites of chemistry 0 will be mobile sites.

    vacsite = cluster.ClusterSite((0, 0), np.zeros(3, dtype=int))
    vacsiteInd = superBCC.index(np.zeros(3, dtype=int), (0, 0))[0]
    assert vacsiteInd == 0

    clusexp = cluster.makeclusters(crys, 1.01*a0, MaxOrder)  # Search for 2-body nn clusters
    Tclusexp = cluster.makeclusters(crys, 1.01*a0, MaxOrderTrans)

    VclusExp = Cluster_Expansion.VectorClusterExpansion(superBCC, clusexp, Tclusexp, jnetBCC,
                                                             NSpec, vacsite, MaxOrder,
                                                             MaxOrderTrans)

    print("Done setting up")
    
    # save it for later
    with open("ClusterExpander.pkl", "wb") as fl:
        pickle.dump(VclusExp, fl)

Pre-created Cluster expander found


## Set up the JIT classes for MC and KMC

In [5]:
# First, we have to generate all the arrays
# Lattice gas - set all energies to zero
Nsym = len(VclusExp.SpecClusters)
Energies = np.zeros(Nsym)
KRAEnergies = [np.zeros(len(val)) for (key, val) in VclusExp.KRAexpander.clusterSpeciesJumps.items()]

numSitesInteracts, SupSitesInteracts, SpecOnInteractSites, Interaction2En, numVecsInteracts,\
VecsInteracts, VecGroupInteracts, numInteractsSiteSpec, SiteSpecInterArray, vacSiteInd,\
InteractionIndexDict, InteractionRepClusDict, Index2InteractionDict, repClustCounter,\
Interact2RepClusArray, Interact2SymClassArray\
= VclusExp.makeJitInteractionsData(Energies)

TsInteractIndexDict, Index2TSinteractDict, numSitesTSInteracts, TSInteractSites, TSInteractSpecs, \
jumpFinSites, jumpFinSpec, FinSiteFinSpecJumpInd, numJumpPointGroups, numTSInteractsInPtGroups, \
JumpInteracts, Jump2KRAEng =\
    VclusExp.KRAexpander.makeTransJitData(KRAEnergies)

Done Indexing interactions : 0.685859203338623
Done with chemical and symmetry class data for interactions : 0.5251574516296387
Done with vector and energy data for interactions : 0.9310939311981201


In [6]:
# Make the MC class to sample initial states
N_units = 8
MCJit = MC_JIT.MCSamplerClass(
    numSitesInteracts, SupSitesInteracts, SpecOnInteractSites, Interaction2En, Interact2RepClusArray,
    Interact2SymClassArray, numVecsInteracts, VecsInteracts, VecGroupInteracts, numInteractsSiteSpec,
    SiteSpecInterArray, numSitesTSInteracts, TSInteractSites, TSInteractSpecs, jumpFinSites, jumpFinSpec,
    FinSiteFinSpecJumpInd, numJumpPointGroups, numTSInteractsInPtGroups, JumpInteracts, Jump2KRAEng
)
# Make the KMC class to generate trajectories
siteIndtoR, RtoSiteInd = VclusExp.makeSiteIndToSite()
KMCJit = MC_JIT.KMC_JIT(numSitesInteracts, SupSitesInteracts, SpecOnInteractSites, Interaction2En,
                        Interact2RepClusArray, Interact2SymClassArray, numVecsInteracts, VecsInteracts,
                        VecGroupInteracts, numInteractsSiteSpec, SiteSpecInterArray, numSitesTSInteracts,
                        TSInteractSites, TSInteractSpecs, jumpFinSites, jumpFinSpec, FinSiteFinSpecJumpInd,
                        numJumpPointGroups, numTSInteractsInPtGroups, JumpInteracts, Jump2KRAEng,
                        siteIndtoR, RtoSiteInd, N_units)

## Let's set up a random state with a single vacancy

In [18]:
# Let's make a state
NSpec = 3
vacsiteInd = VclusExp.sup.index(VclusExp.vacSite.R, VclusExp.vacSite.ci)[0]
assert vacsiteInd == 0 == vacSiteInd
del(vacsiteInd)
state = np.zeros(VclusExp.Nsites, dtype=int)
c0 = 0.6
N0 = int(c0*VclusExp.Nsites)

for i in range(N0):
    state[i] = 0
for i in range(N0, VclusExp.Nsites):
    state[i] = 1

# permute it
state = np.random.permutation(state)
# put the vacancy at the origin
state[vacSiteInd] = NSpec - 1
state_orig = state.copy()  # store a copy in case needed later

### Construct the state's symmetry counts explicitly

In [19]:
# %%time
statePrint, StateTotalSym = VclusExp.GetStateSymInfo(state)

### Now swap sites to generate the new state and get symmetry counts for it too

In [24]:
# Let's produce the swapped state explicitly
Ntrials = 1
initSiteList, finSiteList = MC_JIT.DoRandSwap(state, Ntrials, vacSiteInd)
stateSwap = state.copy()
for trialInd in range(initSiteList.shape[0]):
    siteA = initSiteList[trialInd]
    siteB = finSiteList[trialInd]
    temp = stateSwap[siteA]
    stateSwap[siteA] = stateSwap[siteB]
    stateSwap[siteB] = temp

In [25]:
# Now generate the Fingerprint for this swapped state
StateSwapPrint, StateSwapTotalSym = VclusExp.GetStateSymInfo(stateSwap)

In [26]:
# let's verify the obtained fingerprints by explicit cluster translations

# First, for the original state
stateSymFull, stateSiteSymFull = ClusterSymmInfo(state, VclusExp, Nsym)
assert np.array_equal(stateSymFull, StateTotalSym)
assert np.array_equal(statePrint, stateSiteSymFull)

# Now for the swapped state
stateSwapSymFull, stateSwapSiteFull = ClusterSymmInfo(stateSwap, VclusExp, Nsym)
assert np.array_equal(StateSwapPrint, stateSwapSiteFull)
assert np.array_equal(StateSwapTotalSym, stateSwapSymFull)

## Now use the swapping function in MCJit object to update symmetry counts

In [27]:
# Now, let's try to generate the new state with the JIT swapping code
swapTrials = np.zeros((Ntrials, 2), dtype=int)
swapTrials[:, 0] = initSiteList
swapTrials[:, 1] = finSiteList

Offsc = MC_JIT.GetOffSite(state, numSitesInteracts, SupSitesInteracts, SpecOnInteractSites)
StatePrintJit, StateSymCountsJit = statePrint.copy(), StateTotalSym.copy()
En = 0.
MCJit.GetNewRandState(state, Offsc, StatePrintJit, StateSymCountsJit,
                      swapTrials, En)

In [28]:
assert np.array_equal(state, stateSwap)
assert np.array_equal(StateSwapPrint, StatePrintJit)
assert np.array_equal(StateSwapTotalSym, StateSymCountsJit)
assert np.allclose(En, 0)

# Next, we test a KMC trajectory

## Set up a Fresh random state for this

In [21]:
state = np.zeros(VclusExp.Nsites, dtype=int)
c0 = 0.6
N0 = int(c0*VclusExp.Nsites)

for i in range(N0):
    state[i] = 0
for i in range(N0, VclusExp.Nsites):
    state[i] = 1

# permute it
state = np.random.permutation(state)
# put the vacancy at the origin
state[vacSiteInd] = NSpec - 1
state_orig = state.copy()  # store a copy in case needed later

statePrint, StateTotalSym = VclusExp.GetStateSymInfo(state)

In [22]:
# Set vacancy exchange rates
SpecRates = np.array([0.25, 1.])
# Gather the initial state of the trajectory
stateInit = state.copy()
offscInit = MC_JIT.GetOffSite(stateInit, MCJit.numSitesInteracts,
                          MCJit.SupSitesInteracts, MCJit.SpecOnInteractSites)
siteSymCountsInit = statePrint.copy()
SymCountsTotalInit = StateTotalSym.copy()
stateInit.dtype, siteSymCountsInit.dtype, SymCountsTotalInit.dtype

(dtype('int64'), dtype('int64'), dtype('int64'))

In [23]:
# Get the jump site lists
ijList, dxList = VclusExp.KRAexpander.ijList, VclusExp.KRAexpander.dxList
ijList, dxList

(array([  1,   7, 511,  73,   8,  56,  64, 448]),
 array([[ 0.5,  0.5, -0.5],
        [-0.5, -0.5,  0.5],
        [-0.5, -0.5, -0.5],
        [ 0.5,  0.5,  0.5],
        [ 0.5, -0.5,  0.5],
        [-0.5,  0.5, -0.5],
        [-0.5,  0.5,  0.5],
        [ 0.5, -0.5, -0.5]]))

In [24]:
# Let's make a few steps
offscNext = offscInit.copy()
stateNext = stateInit.copy()
siteSymCountsNext = siteSymCountsInit.copy()
symCountsTotalNext = SymCountsTotalInit.copy()

Nsteps = 1
X_steps, t_steps, jmpSelectSteps, jmpFinSiteList = KMCJit.LatGasTraj(stateNext, siteSymCountsNext, 
                                                                     symCountsTotalNext, offscNext, 
                                                                     SpecRates, Nsteps, ijList,
                                                                     dxList, vacSiteInd)

# Translate the final state to bring the vacancy back to the origin
vacSiteNow = np.where(stateNext == NSpec-1)[0][0]
# Since single step, assert that this is a jump site
assert vacSiteNow in ijList
assert vacSiteNow == ijList[jmpSelectSteps[0]]
stateTrans = KMCJit.TranslateState(stateNext, vacSiteInd, vacSiteNow, None)

In [34]:
# Let's try to swap the state manually
assert np.where(stateInit == NSpec-1)[0][0] == vacSiteInd
stateNextSwap = stateInit.copy()
OffScNextSwap = offscInit.copy()
siteSymCountsNextSwap = siteSymCountsInit.copy()
symCountsTotalNextSwap = SymCountsTotalInit.copy()
siteA = vacSiteInd
siteB = ijList[jmpSelectSteps[0]]
NswapTrials = 1
swapTrials = np.array([[siteA, siteB]])
stateNextSwap = MCJit.GetNewRandState(stateNextSwap, OffScNextSwap, siteSymCountsNextSwap,
                                      symCountsTotalNextSwap, swapTrials, 0.0)

In [28]:
# Now, let's try to translate with fingerprint update
stateTrans = KMCJit.TranslateState(stateNext, vacSiteInd, vacSiteNow, siteSymCountsNext)

In [30]:
np.array_equal(symCountsTotalNextSwap, symCountsTotalNext)

True

In [31]:
# Now for stateTrans, compute the siteSymCounts manually
symCountsNextTransDirect = np.zeros((VclusExp.Nsites, Nsym), dtype=int)
offScTrans = MC_JIT.GetOffSite(stateTrans, numSitesInteracts, SupSitesInteracts, SpecOnInteractSites)
# Go through interactions and get their offcounts
for interactInd in range(offScTrans.shape[0]):
    # Check if the interaction is on
    if offScTrans[interactInd] == 0:
        # get the symmetry class of the interaction
        symclassInteract = Interact2SymClassArray[interactInd]
        # Go through it's sites
        for intSiteInd in range(numSitesInteracts[interactInd]):
            # Get the site
            supSite = SupSitesInteracts[interactInd, intSiteInd]
            # update the symmetry count
            symCountsNextTransDirect[supSite, symclassInteract] += 1

In [32]:
np.array_equal(symCountsNextTransDirect, siteSymCountsNext)

True

# Additional test(s)

### generate finger prints using the JIT arrays

In [ ]:
# Now we first need to calculate the total symcounts
# Also check Offsitecounts here
Offsc = MC_JIT.GetOffSite(state, numSitesInteracts, SupSitesInteracts, SpecOnInteractSites)
StateSymCountsJit = np.zeros(Nsym, dtype=int)
StatePrintJit = np.zeros((VclusExp.Nsites, Nsym), dtype=int)

for interactInd in range(numSitesInteracts.shape[0]):
    numSites=numSitesInteracts[interactInd]
    offCount = 0
    for intSiteInd in range(numSites):
        intSite = SupSitesInteracts[interactInd, intSiteInd]
        intSpec = SpecOnInteractSites[interactInd, intSiteInd]
        if state[intSite] != intSpec:
            offCount += 1
    assert Offsc[interactInd] == offCount
    if offCount == 0:
        # Get the symmetry class of this cluster
        symClass = Interact2SymClassArray[interactInd]
        repClus = VclusExp.Num2Clus[Interact2RepClusArray[interactInd]]
        assert symClass == VclusExp.clust2SpecClus[repClus][0]
        StateSymCountsJit[symClass] += 1
        for intSiteInd in range(numSites):
            intSite = SupSitesInteracts[interactInd, intSiteInd]
            intSpec = SpecOnInteractSites[interactInd, intSiteInd]
            StatePrintJit[intSite, symClass] += 1

In [ ]:
assert np.array_equal(StateTotalSym, StateSymCountsJit)

In [ ]:
assert np.array_equal(StatePrintJit, statePrint)